In [1]:
import os
import numpy as np
import pandas as pd


In [2]:
augmented_dir = './data/_augmented_features' # we will get names from the augmented proteins
augmented_proteins = [x[:-4] for x in os.listdir(augmented_dir) if x[-3:] == 'csv'] # get protein name list to be processed for building machine learning models
print('the number of initial proteins:', len(augmented_proteins))
print(augmented_proteins[:10])

the number of initial proteins: 121
['24622_2', 'A0A024RAY2_P05783', 'A2ABU4', 'A2AHJ4', 'A2AKB9', 'A2AQ25', 'E9K9Z1', 'E9Q1P8', 'E9Q5G3', 'O08537']


# build amino acid sequence dataset

In [3]:
oglcnac_data = pd.read_csv('./data\oglcnacome_sites.csv', index_col=0) 
print(oglcnac_data.info())
oglcnac_data.sample(10, random_state=0)

<class 'pandas.core.frame.DataFrame'>
Index: 4501 entries, 0 to 4500
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   UniprotKB ID   4501 non-null   object 
 1   organism       4501 non-null   object 
 2   oglcnacscore   4501 non-null   float64
 3   oglcnac sites  4501 non-null   object 
 4   sequence       4501 non-null   object 
dtypes: float64(1), object(4)
memory usage: 211.0+ KB
None


,UniprotKB ID,organism,oglcnacscore,oglcnac sites,sequence
1412,P53621,Homo sapiens,22.646970,"[489, 821]",MLTKFETKSARVKGLSFHPKRPWILTSLHNGVIQLWDYRMCTLIDK...
1496,P61019,Homo sapiens,11.106489,[121],MAYAYLFKYIIIGDTGVGKSCLLLQFTDKRFQPVHDLTIGVEFGAR...
2091,Q2T9K0,Homo sapiens,11.981591,"[369, 371]",MGEAPSPAPALWDWDYLDRCFARHRVCISFGLWICASSCWIAAHAL...
3401,Q92993,Homo sapiens,13.761853,[119],MAEVGEIIEGCRLPVLRRNQDNEDEWPLAEILSVKDISGRKLFYVH...
4049,Q9NSY1,Homo sapiens,13.155974,[367],MKKFSRMPKSEGGSGGGAAGGGAGGAGAGAGCGSGGSSVGVRVFAV...
3714,Q9BVC6,Homo sapiens,7.447188,[55],MAASSISSPWGKHVFKAILMVLVALILLHSALAQSRRDFAPPGQQK...
3333,Q91Z67,Mus musculus,20.655088,[990],MTSPAKFKKDKEIIAEYDTQVKEIRAQLTEQMKCLDQQCELRVQLL...
1389,P51957,Homo sapiens,13.756163,[766],MPLAAYCYLRVVGKGSYGEVTLVKHRRDGKQYVIKKLNLRNASSRE...
1922,Q14584,Homo sapiens,6.005927,"[208, 308]",MLENYKNLATVGYQLFKPSLISWLEQEESRTVQRGDFQASEWKVQL...
3352,Q92542,Homo sapiens,13.163917,"[419, 437, 445, 505, 708]",MATAGGGSGADPGSRGLLRLLSFCVLLAGLCRGNSVERKIYIPLNK...


In [4]:
def sequence_with_positivity(protein_data):
    '''
    protein_data: dataframe with single row
    
    '''
    df = pd.DataFrame([x for x in protein_data['sequence'].values[0]], columns=['residue']) 
    
    df['positivity'] = 0
    positive_sites = eval(protein_data['oglcnac sites'].values[0])
    for site in positive_sites:
        df.loc[site-1, 'positivity'] = 1    
        
    return df

In [5]:
'''
augmented protein names include either single element like 'P53621' or multiple elements like 'A0A024RAY2_P05783'
first step is to check if name elements exist in the o-glcnacome database column 'UniprotKB ID' 
if any element exists in the database, then build a dataset of the protein's amino acid sequence with its positive locations
'''

# Convert UniprotKB IDs to a set for faster lookup
database_proteins = set(oglcnac_data['UniprotKB ID'].values)

# Initialize dictionaries and list
positivity_data = {}  # "protein name : sequence with positive sites"
name_augmented_oglcnacome = {}  # names between augmented proteins and database
not_in_database = []  # proteins not in the database

for protein_name in augmented_proteins:
    name_elements = protein_name.split('_')  # Split protein names by '_'
    
    for element in name_elements:
        if element in database_proteins:
            oglcnac_name = element
            break
        else:
            oglcnac_name = None
    
    if oglcnac_name: 
        # Update name matching if protein name has multiple elements
        if len(name_elements) > 1:
            name_augmented_oglcnacome[protein_name] = oglcnac_name
        
        # Retrieve sequence data and update dataset
        protein_oglcnac = oglcnac_data[oglcnac_data['UniprotKB ID'] == oglcnac_name]
        positivity_data[protein_name] = sequence_with_positivity(protein_oglcnac)
    else:
        not_in_database.append(protein_name)

print(f'total number of matching proteins: {len(positivity_data)}')

total number of matching proteins: 112


In [6]:
print('augmented proteins : o-glcnacome database')
display(name_augmented_oglcnacome)

augmented proteins : o-glcnacome database


{'A0A024RAY2_P05783': 'P05783',
 'P0CG62_P0CG49': 'P0CG49',
 'P24622_2': 'P24622',
 'P63249_P63248': 'P63248',
 'P68406_P24622_2': 'P24622',
 'Q4R561_P60710': 'P60710',
 'Q9WVB1_P35279': 'P35279'}

In [7]:
print('proteins not in o-glcnacome database')
print(not_in_database)

proteins not in o-glcnacome database
['24622_2', 'E9K9Z1', 'O08984', 'P02470', 'P02488', 'P02505', 'P04799', 'P05451', 'P07756']


## set variables

In [8]:
from functions import *

In [9]:
for_onehot = { # column_name : classes
    # for input variables
    'residue' : ['A', 'R', 'N', 'D', 'C',
                 'E', 'Q', 'G', 'H', 'I',
                 'L', 'K', 'M', 'F', 'P',
                 'S', 'T', 'W', 'Y', 'V'],
    
    # for output variables
    'positivity' : [0, 1]
}

x_cts = []
x_cat = ['residue']
x_var = x_cts + x_cat

y_cts = []
y_cat = ['positivity']
y_var = y_cts + y_cat

## hyper parameter optimization by K-fold cross-validation

In [10]:
import tensorflow as tf
import keras
from keras.optimizers import Adam
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import KFold

import time
from IPython.display import clear_output

from ml_models import *

epochs = 1000
from keras.callbacks import EarlyStopping
patience = 30
callbacks = [EarlyStopping(patience=patience, restore_best_weights=True, monitor='val_loss')]

SEED = 42
test_size = 0.2

initial_params = {
    'window_size'  : 10,
    'dnn_layers'   : 3,
    'dnn_neurons'  : 64,
    'activation'   : 'softmax',
    'loss'         : 'categorical_crossentropy',
    'metrics'      : 'accuracy',
    'optimizer'    : Adam(learning_rate = 0.001, beta_1=0.9, beta_2=0.999),
    'regularizer'  : {'input': None, 'hidden': None, 'bias': None}
}

search_space = {
    'dnn_layers'  : [1, 2, 3, 4, 5],
    'dnn_neurons' : [32, 64, 128, 256]
}

In [11]:
def name_model(algorithm_type = '', params = {}):
    name = algorithm_type
    for key, value in params.items():
        if key == 'optimizer':
            if type(value) == keras.optimizers.optimizer_v2.adam.Adam:
                name += f"_adam"
            else:
                name += f"_else"
        elif key == 'regularizer':
            name += f"_{value['input']}_{value['hidden']}_{value['bias']}"
            
        else:
            name += f"_{value}"
        
    return name

### K-fold cross-validation

In [12]:
keras.utils.set_random_seed(SEED)
tf.config.experimental.enable_op_determinism()

update = False
            
params = initial_params.copy()
MODELs = []
METRICs = []
METRIC_MEAN = []
model_id = 1
verbose = 0
for param_name, space in search_space.items():
    for point in space:
        clear_output(wait=True)
        display(METRIC_MEAN)
        params[param_name] = point
        print(params)
        
        data_x = []
        data_y = []
        for protein_name in positivity_data:
            positivity = positivity_data.get(protein_name)
            ST_idx = np.where((positivity['residue'] == 'S') | (positivity['residue'] == 'T'))[0]
            
            # get X dataset
            x_onehot = get_onehots(positivity[x_var], columns = x_cat, for_onehot = for_onehot)
            x_features = list(x_onehot.columns)
            
            # get Y dataset
            y_onehot = get_onehots(positivity[y_var], columns = y_cat, for_onehot = for_onehot)
            y_labels = list(y_onehot.columns)
            
            for idx in ST_idx:
                window_x = np.array(get_window(x_onehot, idx, params['window_size']))
                window_x = window_x.reshape(-1)
                label_y  = np.array(y_onehot.iloc[idx])
                
                data_x.append(window_x)
                data_y.append(label_y)
                
        data_x = np.array(data_x)
        data_y = np.array(data_y)

        print('data x shape:', data_x.shape)
        print('data y shape:', data_y.shape)
        print('class y counts:', data_y.sum(0))
        print(f'class y ratio: {(data_y.sum(0)/len(data_y)).round(4)}')
        
        splitter = StratifiedShuffleSplit(n_splits = 1, test_size = test_size, random_state = SEED)
        train_idx, test_idx = list(splitter.split(data_x, data_y))[0]
        
        train_x = data_x[train_idx]
        train_y = data_y[train_idx]
        
        test_x = data_x[test_idx]
        test_y = data_y[test_idx]
        
        splitter_kf = KFold(n_splits = 5)
        scores = []
        for cv_idx, (train_idx_kf, test_idx_kf) in enumerate(splitter_kf.split(train_x, train_y)):
            train_x_kf, train_y_kf = train_x[train_idx_kf], train_y[train_idx_kf]
            test_x_kf, test_y_kf = train_x[test_idx_kf], train_y[test_idx_kf]
            
            model = MLP(data_x.shape[-1], data_y.shape[-1], params)
            model_name = name_model('MLP_KFOLD', params)
            
            
            model_folder  = f'./models/{model_name}_{train_x.shape}_{test_x.shape}'
            if not os.path.exists(model_folder):
                os.makedirs(model_folder)
            model_path    = f'{model_folder}/{cv_idx}.h5'
            metric_path   = f'{model_folder}/{cv_idx}.csv'
            
            
            if not os.path.exists(model_path) or update:
                time_start = time.time()
                history = model.fit(train_x_kf, train_y_kf, verbose=verbose, 
                                    epochs = 10000, callbacks = callbacks,
                                    validation_data = (test_x_kf, test_y_kf))
                time_end = time.time()
                training_time = round((time_end - time_start)/60, 3)
                
                model.save_weights(model_path)
                
                test_loss, accuracy, precision, recall, f1 = metrics_classification(model, test_x, test_y)
                model_metrics = {
                    'model_id' : model_id,
                    'cv_idx'   : cv_idx,
                    'train_x'  : train_x.shape[0],
                    'test_x'   : test_x.shape[0],
                    **params,
                    'regularizer_input' : params['regularizer']['input'],
                    'regularizer_hidden' : params['regularizer']['hidden'],
                    'regularizer_bias' : params['regularizer']['bias'],
                    'training_time': training_time,
                    'test_loss': test_loss,
                    'accuracy': accuracy,
                    **{f'precision_{x}': precision[x] for x in range(len(precision))},
                    **{f'recall_{x}'   : recall[x] for x in range(len(recall))},
                    **{f'f1_{x}'       : f1[x] for x in range(len(f1))}}
                
                model_metrics = pd.DataFrame([model_metrics]).drop(['activation', 'loss', 'metrics', 'optimizer', 'regularizer'], axis=1)
                model_metrics.to_csv(metric_path, index=False)
                
            else:
                model.load_weights(model_path)
                model_metrics = pd.read_csv(metric_path, header=0)
                
            print(f'f1 score: {model_metrics.f1_1[0]}')
            
            model_metrics['model_id'] = model_id
            METRICs.append(model_metrics)
            MODELs.append(model)
        
        METRIC_MEAN = pd.concat(METRICs).groupby('model_id').mean()
        f1_best = METRIC_MEAN.sort_values('f1_1', ascending=False).iloc[0].f1_1
        print(f'best f1 score: {f1_best}')
        model_id += 1
        
    params[param_name] = int(METRIC_MEAN.sort_values('f1_1', ascending=False).iloc[0][param_name])
        

,cv_idx,train_x,test_x,window_size,dnn_layers,dnn_neurons,regularizer_input,regularizer_hidden,regularizer_bias,training_time,test_loss,accuracy,precision_0,precision_1,recall_0,recall_1,f1_0,f1_1
model_id,,,,,,,,,,,,,,,,,,
1,2.0,9696.0,2424.0,10.0,1.0,64.0,NaN,NaN,NaN,0.3318,0.149804,96.290,96.298,20.0,99.992,0.222,98.110,0.440
2,2.0,9696.0,2424.0,10.0,2.0,64.0,NaN,NaN,NaN,0.3460,0.148708,96.298,96.298,20.0,100.000,0.222,98.114,0.440
3,2.0,9696.0,2424.0,10.0,3.0,64.0,NaN,NaN,NaN,0.3640,0.147892,96.290,96.290,0.0,100.000,0.000,98.110,0.000
4,2.0,9696.0,2424.0,10.0,4.0,64.0,NaN,NaN,NaN,0.3556,0.148595,96.290,96.290,0.0,100.000,0.000,98.110,0.000
5,2.0,9696.0,2424.0,10.0,5.0,64.0,NaN,NaN,NaN,0.3622,0.146802,96.290,96.290,0.0,100.000,0.000,98.110,0.000
6,2.0,9696.0,2424.0,10.0,1.0,32.0,NaN,NaN,NaN,0.3186,0.150226,96.274,96.298,10.0,99.976,0.222,98.102,0.434
7,2.0,9696.0,2424.0,10.0,1.0,64.0,NaN,NaN,NaN,0.3318,0.149804,96.290,96.298,20.0,99.992,0.222,98.110,0.440
8,2.0,9696.0,2424.0,10.0,1.0,128.0,NaN,NaN,NaN,0.3110,0.150322,96.298,96.298,20.0,100.000,0.222,98.114,0.440


{'window_size': 10, 'dnn_layers': 1, 'dnn_neurons': 256, 'activation': 'softmax', 'loss': 'categorical_crossentropy', 'metrics': 'accuracy', 'optimizer': <keras.optimizers.optimizer_v2.adam.Adam object at 0x000002B87976ABC0>, 'regularizer': {'input': None, 'hidden': None, 'bias': None}}
data x shape: (12120, 420)
data y shape: (12120, 2)
class y counts: [11672   448]
class y ratio: [0.963 0.037]
f1 score: 2.1999999999999997


c:\Users\honsu\anaconda3\envs\python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


f1 score: 0.0


c:\Users\honsu\anaconda3\envs\python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


f1 score: 0.0


c:\Users\honsu\anaconda3\envs\python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


f1 score: 0.0
f1 score: 0.0
best f1 score: 0.44000000000000006


c:\Users\honsu\anaconda3\envs\python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## model training